<a href="https://colab.research.google.com/github/Daivar/Deep_Learning_Models/blob/main/Bag_of_words_and_text_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# list of text documents
text = ["The quick brown fox jumped over the lazy dog."]
# create the transform
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(text)
# summarize
print(vectorizer.vocabulary_)
# encode document
vector = vectorizer.transform(text)

In [ ]:
print(vector.toarray())

In [ ]:
# summarize encoded vector
print(vector.shape)
print(type(vector))
print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())

In [ ]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
print(list(train_data[0]))

In [ ]:
train_labels[0]


In [ ]:
train_labels[1]

In [ ]:
print(len(list(train_data[1])))

In [ ]:
from keras_preprocessing import sequence
max([max(sequence) for sequence in train_data])

In [ ]:
# word_index is a dictionary mapping words to an integer index
word_index = imdb.get_word_index()
# We reverse it, mapping integer indices to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
# We decode the review; note that our indices were offset by 3
# because 0, 1 and 2 are reserved indices for "padding", "start of sequence", and "unknown".
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in train_data[0]])

In [ ]:
decoded_review

In [ ]:
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

# Our vectorized training data
x_train = vectorize_sequences(train_data)
# Our vectorized test data
x_test = vectorize_sequences(test_data)

In [ ]:
print(list(x_train[0]))
print(list(x_train[1]))
print(len(list(x_train[1])))

In [ ]:
# Our vectorized labels
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

print(y_test[0])
print(y_test[1])
print(y_test[2])

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras import optimizers

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras import losses
from keras import metrics

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.001), loss=losses.binary_crossentropy, metrics=[metrics.binary_accuracy])

In [ ]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [ ]:
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

### overfitinantis modelis, padetų dropautas, LR ratas, epocų apribojimas, 

In [ ]:
plt.clf()   # clear figure
acc_values = history_dict['binary_accuracy']
val_acc_values = history_dict['val_binary_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=4, batch_size=512)
results = model.evaluate(x_test, y_test)

### batch sizas milžiniškas, nepasiekime ACC, mažinti iki 64

In [ ]:
results

In [ ]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras import models
from keras import layers
import numpy as np

from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)
def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

model = models.Sequential()
model.add(layers.Dense(8, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

scores = cross_val_score(KerasClassifier(build_fn=lambda: model, epochs=1, batch_size=10, verbose=1), x_test, y_test, cv=4)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print('Maximum score in cross valuation is: %0.2f' % (scores.max()))

### Against a Random Forest

In [ ]:
from keras.datasets import imdb
import numpy as np

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    # Create an all-zero matrix of shape (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # set specific indices of results[i] to 1s
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

forest_model = RandomForestClassifier(max_depth=50, random_state=1)
# forest_model.fit(x_train, y_train)
# preds = forest_model.predict(x_test)
# preds

In [ ]:
from sklearn.model_selection import cross_val_score

# scores = cross_val_score(RandomForestClassifier(max_depth=50, random_state=1), x_test, y_test, cv=4)
scores = cross_val_score(forest_model, x_test, y_test, cv=4)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print('Maximum score in cross valuation is: %0.2f'%(scores.max()))

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score
results = confusion_matrix(y_test, forest_model.predict(x_test).round())
results

# Deep learning confusion matrixs
# array([[11318,  1182],
#       [  411, 12089]])

# array([[11560,   940],
#       [  582, 11918]])

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(results)

plt.colorbar()
plt.show()

In [ ]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

word_index = imdb.get_word_index()

In [ ]:
def convert_encoded_to_string(data):
    review = []
    for given_value in data:
        for word, value in word_index.items():
            if value + 3 == given_value:
                review.append(word)
    return ' '.join(review)

def convert_string_to_encoded(data):
    review = []
    for given_word in data.split():
        for word, value in word_index.items():
            if word == given_word:
                review.append(value + 3)
    return review

import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

from keras import models
from keras import layers
from keras import regularizers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

from keras.utils.np_utils import to_categorical
model.fit(x_train, to_categorical(y_train), epochs=4, batch_size=512,
          validation_data=(x_test, to_categorical(y_test)))

model.predict(x_test)

custom_review1 = ('this is the best movie i have ever seen in my life '
                  'amazing roles played by the characters this totally '
                  'reminds me of the actual novel which is a must read '
                  'as well')

custom_review2 = ('this film was a garbage the story was meaningless '
                  'and felt bad it was so boring that i felt i was '
                  'being forced to watch this utter useless show')

custom_review3 = ('the movie was fantastic, extraordinary every moment'
                  'of the movie was worth it. it was very enjoyable')

custom_reviews = (custom_review1, custom_review2, custom_review3)
encoded_custom_reviews = np.array([convert_string_to_encoded(review) for review in custom_reviews])
vectorized_custom_reviews = vectorize_sequences(encoded_custom_reviews)

prediction = model.predict(vectorized_custom_reviews)
print(prediction)


# This model gives the output of the 1st layer of the
# original model for some given data

# micro_model = models.Sequential()
# micro_model.add(model.layers[0])
# intermediate_prediction = micro_model.predict(vectorized_custom_reviews)
# print(intermediate_prediction)